This code allows the visualisation of SHAP values for interpretability measures

In [1]:
import shap
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn.model_selection import train_test_split
import tensorflow as tf
import keras
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences

IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html


In [2]:
df = pd.read_csv("../Data_Analysis_on_sentences/IMDB_sentences.csv")[0:1000:1]
model = keras.models.load_model("../CNN_Non_Dense")

tokenizer = Tokenizer()
tokenizer.fit_on_texts(df.Sentences)
tokenized_texts = tokenizer.texts_to_sequences(df.Sentences)
padded_texts = pad_sequences(tokenized_texts, padding="post", maxlen=1000)


Training the SHAP model

In [3]:
explainer = shap.Explainer(model.predict, padded_texts[0:250:1], max_evals=2001)
explainer.max_evals = 10000
# print(padded_texts[0])
first_explanation = explainer(padded_texts[0:2:1])

 22/149 [===>..........................] - ETA: 0s

2023-10-17 23:50:50.969373: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


149/149 [==============================] - 1s 7ms/step


Permutation explainer:  50%|█████     | 1/2 [00:00<?, ?it/s]

115/115 [==============================] - 1s 7ms/step


Permutation explainer: 3it [00:27, 13.73s/it]               


Creating an object to store explanations vs shap values

In [4]:
first_shaps = first_explanation[0].values
word_list = list(df.iloc[0].Sentences.split(sep=" "))
word_vs_shap = list() # list containing the respective word and its shapley additive explanation value

for i in range(len(word_list)):
    word_vs_shap.append({"Word": word_list[i], "SHAP": first_shaps[i]})

# sorting it so we can visualise it easily

def bubbleSort(arr):
    n = len(arr)
    # optimize code, so if the array is already sorted, it doesn't need
    # to go through the entire process
    swapped = False
    # Traverse through all array elements
    for i in range(n-1):
        # range(n) also work but outer loop will
        # repeat one time more than needed.
        # Last i elements are already in place
        for j in range(0, n-i-1):
 
            # traverse the array from 0 to n-i-1
            # Swap if the element found is greater
            # than the next element
            if arr[j]["SHAP"] < arr[j + 1]["SHAP"]:
                swapped = True
                arr[j], arr[j + 1] = arr[j + 1], arr[j]
         
        if not swapped:
            # if we haven't needed to make a single swap, we 
            # can just exit the main loop.
            return
sorted_word_vs_shap = bubbleSort(word_vs_shap)
print(word_vs_shap)


[{'Word': 'has', 'SHAP': 0.07355170280566185}, {'Word': 'episode', 'SHAP': 0.04669677018243714}, {'Word': 'the', 'SHAP': 0.024299972482939986}, {'Word': 'mentioned', 'SHAP': 0.018427379910522172}, {'Word': 'reviewers', 'SHAP': 0.0071822783882631485}, {'Word': '1', 'SHAP': 0.004611469485578993}, {'Word': 'be', 'SHAP': -0.001872659689997865}, {'Word': 'One', 'SHAP': -0.012602398639193003}, {'Word': 'that', 'SHAP': -0.015150573688211185}, {'Word': 'just', 'SHAP': -0.021365733130425113}, {'Word': 'Oz', 'SHAP': -0.02817006213590569}, {'Word': 'of', 'SHAP': -0.03450816942661048}, {'Word': 'hooked', 'SHAP': -0.08781781074401579}, {'Word': 'other', 'SHAP': -0.09439156273008607}, {'Word': 'after', 'SHAP': -0.10192505101922676}, {'Word': 'watching', 'SHAP': -0.10442404497560942}, {'Word': "you'll", 'SHAP': -0.19786665668106215}]


Visualising the SHAP values

In [5]:
import tkinter as tk
from tkinter import *
from tkinter import ttk
max_size=64

root = tk.Tk()
frm = ttk.Frame(root, padding=10)
frm.grid()
frm.rowconfigure(max_size*3) 
frm.columnconfigure(max_size)
sorted_word_vs_shap = word_vs_shap
layer = 1; position = 0
counter = 0
l = [32,16,48,8,24,40,54]
for i in range(0, 7):
    if(i != 0):
        if (counter == 1):
            position += int(max_size / pow(2, layer-4))
            counter = 0
        else:
            position += int(max_size / pow(2, layer-2))
            counter += 1        
    else:
        position += 32
    
    if(i > pow(2, layer-2)):
        if(counter == 0):
            position = int(max_size / pow(2, layer))
        else:
            position = int(max_size / pow(2, layer+1))
        layer += 2
        counter = 0
    

    print("i: {} position: {} layer: {}".format(i, position, layer))
    # the starting position is max_size / pow(2, layer)
    ttk.Label(root, text=round(sorted_word_vs_shap[i]["SHAP"], 5), width=30, wraplength=100, justify="center", font=("Arial", 14)).grid(column=l[i], row=layer)
    ttk.Label(root, text=sorted_word_vs_shap[i]["Word"], width=30, wraplength=100, justify="center", font=("Arial", 14)).grid(column=l[i], row=layer+1)


# root.mainloop()

i: 0 position: 32 layer: 1
i: 1 position: 16 layer: 3
i: 2 position: 48 layer: 3
i: 3 position: 8 layer: 5
i: 4 position: 16 layer: 5
i: 5 position: 48 layer: 5
i: 6 position: 56 layer: 5


In [6]:
root.mainloop()

: 